In [52]:
import pandas as pd
import ta
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import talib

In [53]:
def plot_indicators(data, indicators):
    fig = go.Figure(
        data=[
            go.Candlestick(
                x=data['Datetime'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close']
            ),
            go.Scatter(
                x=data['Datetime'],
                y=indicators
            )
        ]
    )

    fig.show()

In [54]:
data = pd.read_csv('5minutes-kucoin.csv')
data.head()


,Datetime,Open,High,Low,Close,Adj Close,Volume
0,1631577600,44938.3,44964.0,44969.3,44833.0,21.423385,9.618020e+05
1,1631577900,44963.9,44905.7,45035.1,44818.0,14.078011,6.326026e+05
2,1631578200,44905.3,45124.8,45124.9,44890.2,14.729150,6.633849e+05
3,1631578500,45125.0,45027.0,45298.6,45027.0,28.338504,1.280600e+06
4,1631578800,45027.1,44936.4,45098.3,44857.4,21.103995,9.490182e+05


In [55]:
data['MA'] = ta.trend.sma_indicator(close=data["Close"], window=1)
plot_indicators(data, data['MA'].values)

In [56]:
close = data['Close']
window_fast = 1
window_slow = 5
window_sign = 1
data['macd'] = ta.trend.macd(close, window_fast, window_slow, window_sign)
plot_indicators(data, data['macd'].values)

fig = go.Figure(data=[
    go.Scatter(x=data['Datetime'], y=data['macd'].values)
])

fig.show()

In [32]:
close = data['Close']
window = 20
window_dev = 2
indicator_bb = ta.volatility.BollingerBands(close=close, window=window, window_dev=window_dev)

# Add Bollinger Bands features
data['bb_bbm'] = indicator_bb.bollinger_mavg()
data['bb_bbh'] = indicator_bb.bollinger_hband()
data['bb_bbl'] = indicator_bb.bollinger_lband()

# Add Bollinger Band high indicator
data['bb_bbhi'] = indicator_bb.bollinger_hband_indicator()

# Add Bollinger Band low indicator
data['bb_bbli'] = indicator_bb.bollinger_lband_indicator()

In [33]:
plot_indicators(data, data['bb_bbm'].values)

In [34]:
plot_indicators(data, data['bb_bbh'].values)

In [35]:
plot_indicators(data, data['bb_bbl'].values)

In [36]:
plot_indicators(data, data['bb_bbhi'].values)

fig = go.Figure(data=[
    go.Scatter(x=data['Datetime'], y=data['bb_bbhi'].values)
])

fig.show()

In [37]:
plot_indicators(data, data['bb_bbli'].values)

fig = go.Figure(data=[
    go.Scatter(x=data['Datetime'], y=data['bb_bbli'].values)
])

fig.show()

In [39]:
close = data['Close']
window = 2
RSI = ta.momentum.RSIIndicator(close, window)
data['RSI'] = RSI.rsi()
plot_indicators(data, data['RSI'].values)

fig = go.Figure(data=[
    go.Scatter(x=data['Datetime'], y=data['RSI'].values)
])

fig.show()

In [40]:
close = data['Close']
high = data['High']
low = data['Low']
window = 14
smooth_window = 3

Stochastic = ta.momentum.StochasticOscillator(close, high, low, window, smooth_window)
data['Stochastic'] = Stochastic.stoch()

plot_indicators(data, data['Stochastic'].values)

fig = go.Figure(data=[
    go.Scatter(x=data['Datetime'], y=data['Stochastic'].values)
])

fig.show()

In [42]:
close = data['Close']
high = data['High']
low = data['Low']
volume = data['Volume']
window = 14

MFI = ta.volume.MFIIndicator(high, low, close, volume, window)
data['MFI'] = MFI.money_flow_index()

plot_indicators(data, data['MFI'].values)

fig = go.Figure(data=[
    go.Scatter(x=data['Datetime'], y=data['MFI'].values)
])

fig.show()

In [58]:
# Candle Recognitions

doji = talib.CDLDOJI(data['Open'], data['High'], data['Low'], data['Close'])
hammer = talib.CDLHAMMER(data['Open'], data['High'], data['Low'], data['Close'])
idx_doji = np.where(doji != 0)[0]
idx_hammer = np.where(hammer != 0)[0]

fig = go.Figure()

fig.add_trace(go.Candlestick(
        x=data['Datetime'],
        open=data['Open'],
        high=data['High'],
        low=data['Low'],
        close=data['Close']
))

for i in idx_doji:
    fig.add_annotation(
        x=data['Datetime'][i],
        y=data['High'][i],
        showarrow=True,
        text='Doji',
        arrowhead=1
    )

for i in idx_hammer:
    fig.add_annotation(
        x=data['Datetime'][i],
        y=data['High'][i],
        showarrow=True,
        text='Hammer',
        arrowhead=1
    )



fig.update_layout(showlegend=False)


fig.show()

In [59]:
idx_doji

array([], dtype=int64)

In [45]:
# Candle Recognitions

doji = talib.CDLHANGINGMAN(data['Open'], data['High'], data['Low'], data['Close'])
idx = np.where(doji != 0)[0]

fig = go.Figure()

fig.add_trace(go.Candlestick(
        x=data['Datetime'],
        open=data['Open'],
        high=data['High'],
        low=data['Low'],
        close=data['Close']
))

for i in idx:
    fig.add_annotation(
        x=data['Datetime'][i],
        y=data['High'][i],
        showarrow=True,
        text='Hanging Man',
        arrowhead=1
    )


fig.update_layout(showlegend=False)


fig.show()

In [83]:
threshhold = 0.00006

doji_idx = []
for i, item in enumerate(data.iterrows()):
    diff = abs(item[1]['Close'] - item[1]['Open'])
    mean = (item[1]['Close'] + item[1]['Open']) / 2
    if diff < threshhold * (mean):
        print(f'Diff : {diff},  mean : {threshhold * (mean)}')
        doji_idx.append(i)


fig = go.Figure()

fig.add_trace(go.Candlestick(
        x=data['Datetime'],
        open=data['Open'],
        high=data['High'],
        low=data['Low'],
        close=data['Close']
))

for i in doji_idx:
    fig.add_annotation(
        x=data['Datetime'][i],
        y=data['High'][i],
        showarrow=True,
        text='Doji',
        arrowhead=1
    )


fig.update_layout(showlegend=False)


fig.show()

Diff : 2.599999999998545,  mean : 2.702352
Diff : 0.0,  mean : 2.7088080000000003
Diff : 0.0,  mean : 2.7067200000000002
Diff : 0.0,  mean : 2.708016
Diff : 0.09999999999854481,  mean : 2.6998830000000003
Diff : 0.0,  mean : 2.7012419999999997
Diff : 0.6999999999970896,  mean : 2.702331
Diff : 0.0,  mean : 2.70294
Diff : 1.9000000000014552,  mean : 2.713953
Diff : 0.0,  mean : 2.7046740000000002
Diff : 0.09999999999854481,  mean : 2.705997
Diff : 0.0,  mean : 2.711088
Diff : 0.09999999999854481,  mean : 2.713413
Diff : 1.3000000000029104,  mean : 2.713851
Diff : 0.0,  mean : 2.7359459999999998
Diff : 0.0,  mean : 2.738586
Diff : 0.0,  mean : 2.740482
Diff : 0.10000000000582077,  mean : 2.747415
Diff : 0.0,  mean : 2.756166
Diff : 0.0,  mean : 2.757336
Diff : 0.0,  mean : 2.750544
Diff : 0.0,  mean : 2.749926
Diff : 0.09999999999854481,  mean : 2.7498210000000003
Diff : 0.09999999999854481,  mean : 2.7530910000000004
Diff : 0.0,  mean : 2.757432
Diff : 0.0,  mean : 2.75583
Diff : 0.0,  

In [78]:
c = item[1]['Close']
o = item[1]['Open']
h = item[1]['High']
l = item[1]['Low']

C_Len = 14
C_ShadowPercent = 5.0
C_ShadowEqualPercent = 100.0
C_DojiBodyPercent = 5.0
C_Factor = 2.0

C_BodyHi = max(c, o)
C_BodyLo = min(c, o)
C_Body = C_BodyHi - C_BodyLo
C_BodyAvg = 10

[]

In [82]:
a = 46720.97
b = 46723.88
diff = abs(a - b)
mean = (a + b) / 2 
temp = diff / mean
temp

6.228272612126225e-05